Imports and other setup

set_node_attributes was not working for me with networkx 2.0 [see issue here](https://github.com/USEPA/WNTR/issues/29), so downgraded to 1.11 to get it running.

In [21]:
import math
import pandas as pd
import networkx as nx
from visJS2jupyter import visJS_module
import community
import matplotlib.pyplot as plt

# For display purposes
%matplotlib inline
pd.set_option('display.max_rows', 30)
pd.set_option('display.max_columns', 8)
pd.set_option('display.width', 800)

Setup paths, "global" variables, etc

In [22]:
edges_path = '../outputs/edges.csv'
orgs_path = '../outputs/org_names.csv'
output_path = './'
debug = False
# my_nrows = 200 # for testing
my_nrows = None # run on all rows

Read in human readable names, create a usable dict of them

In [23]:
df_org = pd.read_csv(orgs_path, dtype={
'ORGANIZATIONAL_UNIT': int,
'SCHOOL_NAME': object,
'ORG_DISPLAY_NAME': object
})
df_org_names = df_org.set_index('ORGANIZATIONAL_UNIT')
df_org_names = df_org_names[['ORG_DISPLAY_NAME']]
org_names = df_org_names.to_dict()['ORG_DISPLAY_NAME']

Read in edges from csv

In [24]:
if debug:
    if(my_nrows is not None): print "Reading first %d rows of %s" % (my_nrows, edges_path)
    else: print "Reading all rows of %s" % (edges_path)
# need to specify dtypes manually when reading many rows...
# otherwise pandas wants to try to load all the rows into memory before inferring the dtype and you get a warning
df_edges = pd.read_csv(edges_path, dtype={
'Date': object,
'n1': int,
'n2': int
} , nrows=my_nrows)

Build graph G up explicitly, edge by edge. This way we can get the weights correct

In [25]:
G = nx.Graph()
default_weight = 1.0
#default_weight = 1.0 / float(len(df_edges.index))
for index, row in df_edges.iterrows():
    n1 = row['n1']
    n2 = row['n2']
    if G.has_edge(n1,n2):
        G[n1][n2]['weight'] += default_weight
    else:
        G.add_edge(n1,n2, weight=default_weight)

Create position, clusters

In [26]:
spring_pos = nx.spring_layout(G,k=2.5/math.sqrt(nx.number_of_nodes(G))) # 1/sqrt(n) is the default spacing par

In [27]:
parts = community.best_partition(G)
nx.set_node_attributes(G, 'parts', parts) 

Draw interactive graph!

In [28]:
nodes = G.nodes()
node_to_color = visJS_module.return_node_to_color(G,field_to_map='parts',cmap=plt.get_cmap("jet"))
nodes_dict = [{"id":n,
               "title":org_names[n],
              "x":spring_pos[n][0]*1000,
              "y":spring_pos[n][1]*1000,
              #"color":"black"
              "color":node_to_color[n] 
              } for n in nodes]
node_map = dict(zip(nodes,range(len(nodes)))) # map to indices for source/target in edges

In [29]:
edges_list = list(G.edges())
edge_to_color = visJS_module.return_edge_to_color(G,field_to_map='weight',
                                                  cmap=plt.cm.Blues,alpha = 1,color_vals_transform='log')
edges_dict = [{"source":node_map[edges_list[i][0]],
               "target":node_map[edges_list[i][1]],
               "title":"A",
               "edge_title_field":"B",
               "edge_label_field":"C",
               "color":edge_to_color[edges_list[i]] 
              }for i in range(len(edges_list))]

In [20]:
visJS_module.visjs_network(nodes_dict,edges_dict,
                           node_size_multiplier=8,node_font_size=0,
                           edge_width=9,
                           tooltip_delay = 0,
                           graph_width = 900,graph_height = 700,
                           graph_id = 0,
                           config_enabled=False)